In [45]:
import pandas as pd
import spacy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pickle

In [46]:
# Leer CSV
df = pd.read_csv('./data/allData.csv')
df.head()

,content,sentiment
0,@NoilyMV yo soy totalmente puntual,NONE
1,@SandraCauffman Hola Sandrita. No le habia des...,P
2,Si andan haciendo eso mejor se quedaran callad...,N
3,Que pereza quiero choco banano,N
4,"@robertobrenes Bueno, no es tanto lo mayor com...",N


In [47]:
# Sentimientos únicos
pd.DataFrame(df['sentiment'].unique(), columns=['Sentiments Unique'])

,Sentiments Unique
0,NONE
1,P
2,N
3,NEU


In [48]:
# Conversión de sentiments a variables numéricas
df['sentiment'] = df.sentiment.map({'NONE':0, 'N':1, 'NEU':2, 'P':3})

# Vizualización de los sentimientos modificados
df

,content,sentiment
0,@NoilyMV yo soy totalmente puntual,0
1,@SandraCauffman Hola Sandrita. No le habia des...,3
2,Si andan haciendo eso mejor se quedaran callad...,1
3,Que pereza quiero choco banano,1
4,"@robertobrenes Bueno, no es tanto lo mayor com...",1
...,...,...
3670,@AmorAKilates @Roocio_Mk si me pasa lo mismo!,0
3671,@clauchoarrionda pquno ladra y ls demas retwitean,2
3672,A mi desayuno le hizo falta un alfajor podrida...,1
3673,Viste cuando necesitas que alguien te escuche ...,1


In [49]:
# Almacenamiento de stop_words
documents = df['content'].to_list()
nlp = spacy.load("es_core_news_sm")
stopWordsSpanish = []

for text in documents:
    myDoc = nlp(text)
    for word in myDoc:
        if word.is_stop is True and word.text not in stopWordsSpanish:
            stopWordsSpanish.append(word.text)
            #print(word.text)

        if word.like_num is True and word.text not in stopWordsSpanish:
            stopWordsSpanish.append(word.text)
            #print(word.text)

        if word.is_punct is True and word.text not in stopWordsSpanish:
            stopWordsSpanish.append(word.text)
            #print(word.text)

        if any(chr.isdigit() or chr == '_' for chr in word.text) and word.text not in stopWordsSpanish:
            stopWordsSpanish.append(word.text)
            #print(word.text)

stopWordsSpanish

['yo',
 'soy',
 '.',
 'No',
 'le',
 'habia',
 'un',
 'dia',
 'de',
 'la',
 ',',
 'tarde',
 'pero',
 '¿',
 'Sabe',
 'usted',
 'si',
 'ha',
 'mas',
 '?',
 'Si',
 'haciendo',
 'eso',
 'mejor',
 'se',
 'así',
 'están',
 'ellas',
 'mismas',
 'e',
 'que',
 'son',
 'unas',
 'Que',
 'Bueno',
 'no',
 'es',
 'tanto',
 'lo',
 'mayor',
 'como',
 'cuánto',
 'sea',
 'en',
 'cada',
 'tu',
 'por',
 'todo',
 'el',
 'día',
 'mi',
 'y',
 'Estaba',
 'ahí',
 'El',
 'Y',
 'los',
 'fueron',
 'todavía',
 'En',
 'hay',
 'me',
 'van',
 'a',
 'las',
 'Tengo',
 'tengo',
 'QUÉ',
 'HAGO',
 'puedo',
 'más',
 'ni',
 '1',
 '13:4-5',
 'Buen',
 'todos',
 'o2',
 '...',
 '6:00',
 'al',
 'con',
 'ver',
 'una',
 'Lo',
 'peor',
 'tiene',
 '5',
 'HACE',
 'era',
 'creo',
 'mio',
 '!',
 'Casi',
 'ya',
 'Con',
 'del',
 '150kmph',
 '@Colucci_99',
 'Esas',
 'he',
 'hasta',
 'ahora',
 'Antes',
 'hace',
 '..',
 'Buenas',
 'Me',
 '50mil',
 'manera',
 'hacer',
 'toda',
 'esta',
 '12',
 '@josdav17',
 'nunca',
 'hacen',
 'nada',
 'según

In [50]:
# Inicialización del Contador de Vectorización
count_vector = CountVectorizer(stop_words=stopWordsSpanish)

In [51]:
# Lista de palabrás únicas
count_vector.fit(documents)
names = count_vector.get_feature_names()
names

C:\Users\ASUS\Documents\Entornos\Entorno01\lib\site-packages\sklearn\feature_extraction\text.py:404: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['00', '000', '00am', '02', '03', '04', '0hmyvolunteer', '102', '10_nov', '10s', '143', '150', '1559', '1d', '1maaariii1', '204danielle', '20s', '22', '2225', '2820gchu', '2891891cap', '2c', '2tes', '3080', '30am', '30hs', '30pm', '3k', '46pm', '55', '5honamas', '5nfidente', '60', '7000', '74evmm', '80', '87', '8827', '90feliznavidad', '99allison11', '9km', '__el_negro', '__josejuan', '__walter', '_adrianaquevedo', '_alejandrodp', '_alineepa', '_anatabeira', '_cmere', '_fer_alonso', '_fersha', '_josejones', '_mariajosee', '_melibarra', '_monsejimenezp', '_nosoyartista', '_pepepar', '_star_vader', '_suniac', '_void07', 'abbu25', 'abby_maff', 'abiel_vasquez', 'absa_lon', 'acompañarnos', 'adri_hdezv', 'adri_tranqui', 'adriantr09', 'afigue2010', 'agronegocios_uy', 'agusblanco

['00',
 '000',
 '00am',
 '02',
 '03',
 '04',
 '0hmyvolunteer',
 '102',
 '10_nov',
 '10am',
 '10s',
 '11pm',
 '143',
 '150',
 '1559',
 '1cm',
 '1d',
 '1k',
 '1maaariii1',
 '204danielle',
 '20s',
 '22',
 '2225',
 '2820gchu',
 '2891891cap',
 '2c',
 '2pm',
 '2tes',
 '3080',
 '30am',
 '30cm',
 '30hs',
 '30pm',
 '3k',
 '3pm',
 '46pm',
 '4am',
 '4g',
 '4k',
 '55',
 '5honamas',
 '5nfidente',
 '60',
 '6pm',
 '7000',
 '74evmm',
 '7am',
 '80',
 '87',
 '8827',
 '8am',
 '8g',
 '90feliznavidad',
 '99allison11',
 '9am',
 '9km',
 '__',
 '__el_negro',
 '__josejuan__',
 '__walter__',
 '_adrianaquevedo',
 '_alejandrodp',
 '_alineepa',
 '_anatabeira',
 '_cmere',
 '_fer_alonso',
 '_fersha_',
 '_josejones',
 '_mariajosee_',
 '_melibarra',
 '_monsejimenezp',
 '_nosoyartista',
 '_pepepar',
 '_star_vader',
 '_suniac',
 '_tener',
 '_void07',
 'aaa',
 'aaaa',
 'aaaaaaah',
 'aaah',
 'aaahhhhggg',
 'ab',
 'abajo',
 'abandonada',
 'abandonado',
 'abarcara',
 'abbu25',
 'abby_maff',
 'abcdesignated',
 'abdiel',
 'ab

In [52]:
doc_array = count_vector.transform(documents).toarray()
doc_array

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [53]:
frequency_matrix = pd.DataFrame(data=doc_array, columns=names)
frequency_matrix

,00,000,00am,02,03,04,0hmyvolunteer,102,10_nov,10am,...,úmica,única,únicamente,únicas,único,únicos,úrgeme,úselo,útil,útiles
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3670,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3671,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3672,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3673,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [54]:
# Preparación de los datos
X_train, X_test, y_train, y_test = train_test_split(df['content'], df['sentiment'], random_state=1)

In [55]:
# Instanciar el metodo CountVector
count_vector = CountVectorizer()

# Ajuste los datos de entrenamiento y luego devuelva la matriz
training_data = count_vector.fit_transform(X_train)

# Transforme los datos de prueba y devuelva la matriz. Tener en cuenta que no se está ajustando los datos de prueba en CountVectorizer()
testing_data = count_vector.transform(X_test)

In [56]:
# Uso de Máquinas de Vector Soporte (SVM)
clf = svm.SVC()
clf.fit(training_data, y_train)

SVC()

In [57]:
# Predicción del modelo
predictions = clf.predict(testing_data)

In [58]:
# Evaluación del modelo
print('Accuracy score: ', format(accuracy_score(y_test, predictions)))
print('Precision score: ', format(precision_score(y_test, predictions, average = None)))
print('Recall score: ', format(recall_score(y_test, predictions, average = None)))
print('F1 score: ', format(f1_score(y_test, predictions, average = None)))

Accuracy score:  0.5244831338411317
Precision score:  [0.61904762 0.47806354 0.         0.64556962]
Recall score:  [0.07647059 0.9002849  0.         0.56043956]
F1 score:  [0.13612565 0.62450593 0.         0.6       ]


C:\Users\ASUS\Documents\Entornos\Entorno01\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [59]:
# Exportación de los Modelos
pickle.dump(count_vector, open('my_app/modelos/vectorSVM.sav', 'wb'))
pickle.dump(clf, open('my_app/modelos/classifierSVM.sav', 'wb'))